In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
import sys
from pylab import *
import os
import urllib.request
import seaborn as sns

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [ ]:
#The feistiness index represents teams not giving up
#It sums positive EPA plays when win probability is less than 35%
#It does not punish for negative EPA plays
#It is only interested in the punches you land

In [ ]:
YEAR = 2020
data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(YEAR) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

In [ ]:
underdogs = data.loc[(data['wp'] < 0.35) & (data['posteam_score'] < data['defteam_score']) 
& (data['wpa'] > 0)]
underdogs = underdogs[['posteam', 'defteam', 'posteam_score', 'defteam_score', 'wp', 'wpa', 'desc']].sort_values(by='wpa', ascending=False)
underdogs.head(10)
#underdogs.loc[(data.posteam == 'PHI')].head(10)

In [ ]:
wp_vals = underdogs.groupby('posteam')['wpa'].sum()
wp_vals = wp_vals.reset_index()
#wp_vals

In [ ]:
#cols = data.columns.tolist()

In [ ]:
COLORS = {'ARI':'#97233F','ATL':'#A71930','BAL':'#241773','BUF':'#00338D','CAR':'#0085CA','CHI':'#00143F',
          'CIN':'#FB4F14','CLE':'#FB4F14','DAL':'#B0B7BC','DEN':'#002244','DET':'#046EB4','GB':'#24423C',
          'HOU':'#C9243F','IND':'#003D79','JAX':'#136677','KC':'#CA2430','LA':'#002147','LAC':'#2072BA',
          'LV':'#C4C9CC','MIA':'#0091A0','MIN':'#4F2E84','NE':'#0A2342','NO':'#A08A58','NYG':'#192E6C',
          'NYJ':'#203731','PHI':'#014A53','PIT':'#FFC20E','SEA':'#7AC142','SF':'#C9243F','TB':'#D40909',
          'TEN':'#4095D1','WAS':'#FFC20F'}

In [ ]:
#sns.catplot(x='posteam', y='wpa', kind='swarm', data=underdogs, height=10, aspect = 2)

#urls = pd.read_csv('https://raw.githubusercontent.com/statsbylopez/BlogPosts/master/nfl_teamlogos.csv')
#for i in range(0,len(urls)):
#    urllib.request.urlretrieve(urls['url'].iloc[i], os.getcwd() + '\\nfllogos\\' + urls['team'].iloc[i] + '.png')

In [ ]:
   
#this function allows us to get images onto chart
def getImage(path): 
    return OffsetImage(plt.imread(path), zoom=.5)

logos = os.listdir(os.getcwd() + '\\nfllogos')

logo_paths = []

for i in logos:
    logo_paths.append(os.getcwd() + '\\nfllogos\\' + str(i))
    
logo_paths

#Add team colors to dataframe
wp_vals['color'] = COLORS.values()

#Add path column where each row will have the location of the team's logo
wp_vals['path'] = logo_paths

#logo_paths

In [ ]:
fig, ax = plt.subplots(figsize=(30,10))

#sort by feistyness
wp_vals = wp_vals.sort_values(by='wpa', ascending=False)

#Add logos
#X data is an array from 0-31 (for 32 teams)
for x0, y0, path in zip(np.arange(0,32), wp_vals['wpa']+.005,wp_vals['path']):
    ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False, fontsize=4)
    ax.add_artist(ab)

#create bar chart, setting colors for each bar
ax.bar(np.arange(0, 32), wp_vals['wpa'], color=wp_vals.color,width=.5)

#set league average feistiness
ax.axhline(y=wp_vals['wpa'].mean(), linestyle='--', color='grey')

#add grid
ax.grid(zorder=0, alpha=.6, axis='y')
ax.set_axisbelow(True)

#Set x-ticks labels to be team abbreviations
ax.set_xticks(np.arange(0,32))
ax.set_xticklabels(wp_vals.posteam,fontsize=16)

#Set labels and give source
ax.set_ylabel('Feistiness (Sum +WPA when WP Under 35%)',fontsize=20,labelpad=20)
ax.set_title('Feistiness Index Through Week 13, 2020',
             fontsize=26,pad=20)
plt.figtext(.81, 0.06,'Data: nflfastR',fontsize=14)
plt.figtext(.81, 0.03,'Chart: @goodfriendadam',fontsize=14)

#League average line label
plt.text(20, 3.4,'League Average Feistiness',fontsize=14)

plt.savefig(os.getcwd() + '\\output\\feistinessIndex2020.png',dpi=400)